In [ ]:

# 1 Carga librerías

from __future__ import annotations
import argparse
import json
import os
import re
from pathlib import Path
from typing import Dict, List
import accelerate
import pandas as pd
import pdfplumber
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer, pipeline

In [ ]:
# 2 Login Hugging Face
from huggingface_hub import login
login()

In [ ]:
# 3 Prompt usado con el modelo 
PROMPT_TEMPLATE = """
Eres un asistente que extrae información estructurada de currículums vitae (CV) en español.


Devuelve exclusivamente un sólo objeto JSON con las siguientes claves y NINGÚN texto adicional:


- nombre (puede estar en el encabezado)
- domicilio 
- telefono 
- titulo (titulo universitario)
- institucion (institución desde la que se egreso si es universitaria)
- anios_experiencia (respuesta int con un sólo valor suma de los años que van desde el primer trabajo al último)
- cantidad_trabajos (respuesta int con un sólo valor que será mayor a uno si ha habido cambio de trabajo)
- ultimo_empleador


Curriculum del que se extraera la informacion:
"""

JSON_FOLLOWS = "\nJSON:"


In [ ]:

# 4 Funciones usadas

def extract_text_from_pdf(path: Path) -> str:
    """Extrae texto de un PDF completo usando pdfplumber."""
    with pdfplumber.open(path) as pdf:
        text = "\n".join(page.extract_text() or "" for page in pdf.pages)
    return text

def call_llm(resume_text: str) -> Dict[str, str]:
    """Envía el texto del CV al LLM y parsea la respuesta JSON."""
    prompt = PROMPT_TEMPLATE + "\n\n" + resume_text + JSON_FOLLOWS
    raw_output = generate(prompt)[0]["generated_text"]
    
    json_part = raw_output.split(JSON_FOLLOWS, 1)[-1].strip()
    
    # para limpiar posibles saltos de línea y espacios extra
    #    y aislar desde la primera '{' hasta la última '}'
    start = json_part.find("{")
    end = json_part.rfind("}")
    if start != -1 and end != -1:
        json_part = json_part[start : end+1].strip()
    
    #  se intenta parsear JSON; si falla, se reintenta con regex
    try:
        return json.loads(json_part)
    except json.JSONDecodeError:
        match = re.search(r"\{.*\}", json_part, re.S)
        if match:
            return json.loads(match.group(0))
        # Si sigue fallando, devolvemos un dict vacío o podríamos lanzar un error
        return {}


def process_folder(folder_path: Path) -> pd.DataFrame:
    """Procesa todos los PDFs de la carpeta y devuelve un DataFrame."""
    lista_pdf = os.listdir(folder_path) 
    lista_jsons = []
    for pdf in lista_pdf:
        pdf_path = os.path.join(folder_path, pdf)
        text = extract_text_from_pdf(pdf_path)
        info = call_llm(text)
        lista_jsons.append(info)
    return lista_jsons

In [ ]:
# 5 cargo el modelo

MODEL_NAME = "HuggingFaceH4/zephyr-7b-beta"
bnb_config = BitsAndBytesConfig(load_in_4bit=True)


def load_llm():


    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        device_map="auto",              
        offload_folder="offload",       
        offload_state_dict=True,
        quantization_config=bnb_config,
        trust_remote_code=True,
        )

    generate = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=256,
        temperature=0.1,
        return_full_text=False
    )

    return generate

generate = load_llm()


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

c:\Users\Admin\anaconda3\envs\tensorgpu\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Admin\.cache\huggingface\hub\models--HuggingFaceH4--zephyr-7b-beta. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Device set to use cuda:0


In [ ]:
# 6) path con cv sintéticos

path_sintetico = r"C:\Users\Admin\Documents\1_Notebook\2_Trabajo\Github\LLM_CV_Processor\HR-llm-CV-processor\cv_sinteticos"

lista_datos_cv_sintetico = process_folder(path_sintetico)

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
c:\Users\Admin\anaconda3\envs\tensorgpu\lib\site-packages\transformers\generation\configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox mis

In [ ]:
# 8) Ejemplo de respuesta recibida
lista_datos_cv_sintetico[0]

{'nombre': 'Cayetano Pérez',
 'dirección': 'Avenida de Juanita Iglesia 4 Apt. 56',
 'teléfono': '+34874027395',
 'email': 'nietofabiana@gmail.com',
 'linkedin': 'https://www.linkedin.com/in/cayetanoperera',
 'titulo': 'Ingeniería en Sistemas',
 'institucion': 'Ramirez Group University',
 'anios_experiencia': 5,
 'cantidad_trabajos': 1,
 'ultimo_empleador': 'Amorós-Nogués'}

In [ ]:
# 9) etl error común
for json in lista_datos_cv_sintetico:
    lista_claves = list(json.keys())
    if 'teléfono' in lista_claves or 'dirección' in lista_claves:
        for clave in list(json.keys()):  # usamos una copia
            if clave == 'teléfono':
                json['telefono'] = json.pop("teléfono")
            if clave == 'dirección':
                json['direccion'] = json.pop("dirección")

                
                

In [ ]:
# 10 Dataframe final
df_sintetico = pd.DataFrame(lista_datos_cv_sintetico)

In [67]:
df_sintetico

,nombre,email,linkedin,titulo,institucion,anios_experiencia,cantidad_trabajos,ultimo_empleador,direccion,telefono
0,Cayetano Pérez,nietofabiana@gmail.com,https://www.linkedin.com/in/cayetanoperera,Ingeniería en Sistemas,Ramirez Group University,5.0,1.0,Amorós-Nogués,Avenida de Juanita Iglesia 4 Apt. 56,+34874027395
1,Dafne del Solís,juan-manuel10@porta.es,https://www.linkedin.com/in/dalilainfante,Ingeniería en Sistemas,"Salom, Abellán and Morillo University",6.0,1.0,Miralles-Viña,"Acceso de Glauco Juliá 6 Puerta 7, Salamanca, ...",+34 700 543 711
2,Eli Farré-Pintor,casemiroredondo@piquer.es,https://www.linkedin.com/in/maria-joseburgos,Ingeniería en Sistemas,Román Inc University,5.0,1.0,Suárez PLC,"Pasaje María Vélez 90 Puerta 3, Ávila, 79380",+34865064804
3,Esteban Sola Hernandez,iriartetito@hierro-llanos.net,https://www.linkedin.com/in/bartolome73,Ingeniería en Sistemas,"Guijarro, Ramírez and Cueto University",7.0,1.0,Camacho-Abad,"Camino Fausto Peinado 61, Vizcaya, 02426",+34713581111
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Fermín Larrañaga Arias,lorenza85@gmail.com,https://www.linkedin.com/in/guiomarmarin,Ingeniería en Sistemas,"Melero, Requena and Canals University",6.0,1.0,Alvarez-Gilabert,"Pasaje Aitor Milla 66 Apt. 20, Huesca, 48801",+34712167543
6,Héctor Elías-Sanz,guijarroale@hotmail.com,https://www.linkedin.com/in/leocadia07,Ingeniería en Sistemas,"Ribas, Baró and Tamarit University",3.0,1.0,"Salazar, Frutos and Heredia","C. Macarena Tamarit 505, Girona, 50196",+34 724 436 535
7,Isidoro Llanos Giner,marinfelisa@yahoo.com,https://www.linkedin.com/in/aguedaibanez,Ingeniería en Sistemas,"Pino, Prats and Gutiérrez University",4.0,1.0,Hernando Group,"Alameda de Loida Nuñez 6 Piso 0, Salamanca, 94531",+34742650343
8,Juanita Alcántara Pujadas,jonatan25@feijoo-amoros.com,https://www.linkedin.com/in/maura86,Ingeniería en Sistemas,Muro and Sons University,4.0,1.0,"Poza, Sainz and Nogués","Cañada de Anna Zapata 8 Apt. 67, La Coruña, 22906",+34744 289 745
9,Julieta Bustamante Prada,esperanza62@hotmail.com,https://www.linkedin.com/in/barredacoral,Ingeniería en Sistemas,Cánovas PLC University,5.0,1.0,"Fortuny, Sancho and Galan","Urbanización de Abraham Delgado 19, Salamanca,...",+34725 567 076


In [75]:
df_sintetico.to_csv(os.getcwd()+'df_sintetico.csv')